In [1]:
import datetime
print("Current Date/Time:", datetime.datetime.now())

Current Date/Time: 2021-02-06 17:40:44.690370


In [1]:
from google.colab import drive

drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


In [2]:
import torch
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
#!pip install torchsummary
from torchsummary import summary
import pandas as pd
import numpy as np

In [3]:
import sys
sys.path.append ('/content/drive/MyDrive/CVMODEL/GradCam/')
from Quiz_DNN import Net

Data Loading

In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if cuda else "cpu")

CUDA Available? True


In [5]:
common_transforms = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.49186878, 0.48265391, 0.44717728), (0.24697121, 0.24338894, 0.26159259))
     ])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=common_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=common_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
model = Net(dropout=0.1).to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
           Dropout-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           5,472
              ReLU-6           [-1, 32, 32, 32]               0
       BatchNorm2d-7           [-1, 32, 32, 32]              64
           Dropout-8           [-1, 32, 32, 32]               0
         MaxPool2d-9           [-1, 51, 16, 16]               0
           Conv2d-10           [-1, 64, 16, 16]          29,376
             ReLU-11           [-1, 64, 16, 16]               0
      BatchNorm2d-12           [-1, 64, 16, 16]             128
          Dropout-13           [-1, 64, 16, 16]               0
           Conv2d-14          [-1, 128,

/content/drive/MyDrive/CVMODEL/GradCam/Quiz_DNN.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x13)


In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch_number,l1_loss=False, l1_beta = 0):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    train_accuracy = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_accuracy += pred.eq(target.view_as(pred)).sum().item()
        loss = criterion(output, target)
        if l1_loss == True:
            l1_crit = nn.L1Loss(size_average=False)
            reg_loss = 0
            for param in model.parameters():
                target = torch.zeros_like(param)    
                reg_loss += l1_crit(param, target)
            loss += (l1_beta * reg_loss)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)
    train_accuracy = (100. * train_accuracy) / len(train_loader.dataset)
    return train_accuracy, train_loss


In [9]:
def test(model, device, test_loader, epoch_number):
    model.eval()
    test_loss = 0
    correct = 0
    example_images = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = (100. * correct) / len(test_loader.dataset)
    return test_accuracy, test_loss

In [10]:
for epoch in range(1, 39):
        epoch_train_acc,epoch_train_loss = train(model, device, trainloader, optimizer, epoch)        
        epoch_test_acc,epoch_test_loss = test(model, device, testloader,epoch)
        print('\nEpoch: {:.0f} Train set: Average loss: {:.4f}, Accuracy: {:.3f}%'.format(epoch, epoch_train_loss, epoch_train_acc))
        print('Epoch: {:.0f} Test set: Average loss: {:.4f}, Accuracy: {:.3f}%'.format(epoch, epoch_test_loss, epoch_test_acc))


  0%|          | 0/782 [00:00<?, ?it/s]/content/drive/MyDrive/CVMODEL/GradCam/Quiz_DNN.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x13)
  0%|          | 0/782 [00:00<?, ?it/s]


Epoch: 1 Train set: Average loss: 0.0215, Accuracy: 50.820%
Epoch: 1 Test set: Average loss: 0.0199, Accuracy: 55.070%


  0%|          | 0/782 [00:00<?, ?it/s]


Epoch: 2 Train set: Average loss: 0.0159, Accuracy: 64.362%
Epoch: 2 Test set: Average loss: 0.0159, Accuracy: 63.940%


  0%|          | 0/782 [00:00<?, ?it/s]

...


Epoch: 36 Train set: Average loss: 0.0002, Accuracy: 99.698%
Epoch: 36 Test set: Average loss: 0.0096, Accuracy: 85.730%


  0%|          | 0/782 [00:00<?, ?it/s]


Epoch: 37 Train set: Average loss: 0.0002, Accuracy: 99.722%
Epoch: 37 Test set: Average loss: 0.0103, Accuracy: 84.930%


loss=0.15289711952209473 batch_id=781: 100%|██████████| 782/782 [00:40<00:00, 19.30it/s]



Epoch: 38 Train set: Average loss: 0.0002, Accuracy: 99.664%
Epoch: 38 Test set: Average loss: 0.0097, Accuracy: 85.680%
